In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from geopy.geocoders import Nominatim
from math import radians, cos, sin, asin, sqrt
from numpy import cos, sin, arcsin, sqrt
from ast import literal_eval
from tqdm import tqdm
import requests
import json
import networkx as nx
import osmnx as ox
import requests
from geopy import Point
from geopy.distance import geodesic
import numpy as np

In [2]:
df_building = pd.read_csv('MexicoCityBuildingsFinal.csv')

In [3]:
df_building.rename(columns={'name' : 'Building'}, inplace = True)

In [4]:
df_civic = pd.read_csv('CivicDistance.csv')
df_government = pd.read_csv('GovernmentDistance.csv')
df_hospital = pd.read_csv('HospitalDistance.csv')
df_public = pd.read_csv('PublicDistance.csv')
df_school = pd.read_csv('SchoolDistance.csv')
df_sports = pd.read_csv('SportsDistance.csv')
df_supermarket = pd.read_csv('SupermarketDistance.csv')
df_transport = pd.read_csv('TransportDistance.csv')

In [5]:
df_civic = df_civic.groupby('InputID').agg(lambda x: list(x))
df_government = df_government.groupby('InputID').agg(lambda x: list(x))
df_hospital = df_hospital.groupby('InputID').agg(lambda x: list(x))
df_public = df_public.groupby('InputID').agg(lambda x: list(x))
df_school = df_school.groupby('InputID').agg(lambda x: list(x))
df_sports = df_sports.groupby('InputID').agg(lambda x: list(x))
df_supermarket = df_supermarket.groupby('InputID').agg(lambda x: list(x))
df_transport = df_transport.groupby('InputID').agg(lambda x: list(x))

In [6]:
df_civic = df_civic.reset_index()
df_government = df_government.reset_index()
df_hospital = df_hospital.reset_index()
df_public = df_public.reset_index()
df_school = df_school.reset_index()
df_sports = df_sports.reset_index()
df_supermarket = df_supermarket.reset_index()
df_transport = df_transport.reset_index()

In [7]:
df_civic.drop(columns=['TargetID'], inplace = True)
df_government.drop(columns=['TargetID'], inplace = True)
df_hospital.drop(columns=['TargetID'], inplace = True)
df_public.drop(columns=['TargetID'], inplace = True)
df_school.drop(columns=['TargetID'], inplace = True)
df_sports.drop(columns=['TargetID'], inplace = True)
df_supermarket.drop(columns=['TargetID'], inplace = True)
df_transport.drop(columns=['TargetID'], inplace = True)

In [8]:
df_civic
df_fire
df_government
df_hospital
df_public
df_school
df_sports
df_supermarket
df_toilet
df_transport

NameError: name 'df_fire' is not defined

In [9]:
df_civic.rename(columns={'InputID':'Building', 'Distance':'Civic'}, inplace = True)
df_government.rename(columns={'InputID':'Building', 'Distance':'Government'}, inplace = True)
df_hospital.rename(columns={'InputID':'Building', 'Distance':'Hospital'}, inplace = True)
df_public.rename(columns={'InputID':'Building', 'Distance':'Public'}, inplace = True)
df_school.rename(columns={'InputID':'Building', 'Distance':'School'}, inplace = True)
df_sports.rename(columns={'InputID':'Building', 'Distance':'Sports'}, inplace = True)
df_supermarket.rename(columns={'InputID':'Building', 'Distance':'Supermarket'}, inplace = True)
df_transport.rename(columns={'InputID':'Building', 'Distance':'Transport'}, inplace = True)

In [10]:
df = df_building.merge(df_civic, on='Building', how = 'inner')
df = df.merge(df_government, on='Building', how = 'inner')
df = df.merge(df_hospital, on='Building', how = 'inner')
df = df.merge(df_public, on='Building', how = 'inner')
df = df.merge(df_school, on='Building', how = 'inner')
df = df.merge(df_sports, on='Building', how = 'inner')
df = df.merge(df_supermarket, on='Building', how = 'inner')
df = df.merge(df_transport, on='Building', how = 'inner')

In [ ]:
df

In [ ]:
df.columns

In [11]:
df[['Civic', 'Government', 'Hospital', 'Public', 'School', 'Sports', 'Supermarket', 'Transport']] = df[['Civic', 'Government', 'Hospital', 'Public', 'School', 'Sports', 'Supermarket', 'Transport']].applymap(lambda x: np.divide(x, 10000) if isinstance(x, list) else x)

In [12]:
#df[['Civic', 'Government']] = df[['Civic', 'Government']].applymap(lambda x: np.divide(x, 10000) if isinstance(x, list) else x)

In [13]:
distance_map = {
    'Civic': 1.5,
    'Fire': 1.0,
    'Government': 10.0,
    'Hospital': 10.0,
    'Public': 2.0,
    'School': 1.0,
    'Sports': 1.5,
    'Supermarket': 1.0,
    'Toilet': 0.1,
    'Transport': 2.0,
}
for facility, max_dist in distance_map.items():
    try:
        df[f'{facility}_nearby'] = df[facility].apply(lambda lst: [d for d in lst if d <= max_dist])
    except:
        pass


for facility, max_dist in distance_map.items():
    try:
        df[f'{facility}_nearby'] = df[f'{facility}_nearby'].apply(lambda lst: [round(max_dist - d, 2) for d in lst])
    except:
        pass


for facility in distance_map:
    try:
        df[f'{facility}_Score'] = df[f'{facility}_nearby'].apply(sum)
    except:
        pass

from sklearn.preprocessing import MinMaxScaler

# Select score columns
score_cols = [col for col in df.columns if col.endswith('_Score')]

# Initialize scaler with feature range 0-10
scaler = MinMaxScaler(feature_range=(0, 10))

# Fit and transform, then round to 2 decimals
df[score_cols] = pd.DataFrame(
    scaler.fit_transform(df[score_cols]),
    columns=score_cols,
    index=df.index
).round(2)


In [14]:
df.columns

Index(['index', 'Building', 'name:en', 'building', 'building:levels',
       'building:materials', 'addr:full', 'addr:housenumber', 'addr:street',
       'addr:city', 'office', 'source', 'name:es', 'osm_id', 'osm_type',
       'ADM1_ES', 'ADM1_PCODE', 'ADM1_REF', 'ADM1ALT1ES', 'ADM1ALT2ES',
       'ADM0_ES', 'ADM0_PCODE', 'date', 'validOn', 'validTo', 'Shape_Leng',
       'Shape_Area', 'lon', 'lat', 'Civic', 'Government', 'Hospital', 'Public',
       'School', 'Sports', 'Supermarket', 'Transport', 'Civic_nearby',
       'Government_nearby', 'Hospital_nearby', 'Public_nearby',
       'School_nearby', 'Sports_nearby', 'Supermarket_nearby',
       'Transport_nearby', 'Civic_Score', 'Government_Score', 'Hospital_Score',
       'Public_Score', 'School_Score', 'Sports_Score', 'Supermarket_Score',
       'Transport_Score'],
      dtype='object')

In [15]:
df = df[['Building','lon', 'lat', 'Civic_Score', 'Government_Score', 'Hospital_Score',
       'Public_Score', 'School_Score', 'Sports_Score', 'Supermarket_Score',
       'Transport_Score']]

In [16]:
df.to_csv('MexicoCityScoreRAW1.csv', index = False)

In [14]:
%%time
columns = ['Civic', 'Government', 'Hospital', 'Public', 'School', 'Sports', 'Supermarket', 'Transport']

for col in columns:
    df[f'{col}_1km'] = None
    df[f'{col}_5km'] = None
    df[f'{col}_10km'] = None


CPU times: total: 31.2 ms
Wall time: 46.9 ms


In [15]:
%%time
# Define columns
columns = ['Civic', 'Government', 'Hospital', 'Public', 'School', 'Sports', 'Supermarket', 'Transport']

# Define a function to filter numbers based on distance
def filter_distances(row, columns):
    for col in columns:
        row[f'{col}_1km'] = [number for number in row[col] if number <= 1]
        row[f'{col}_5km'] = [number for number in row[col] if number <= 5]
        row[f'{col}_10km'] = [number for number in row[col] if number <= 10]
    return row

# Apply the function across the dataframe
df = df.apply(lambda row: filter_distances(row, columns), axis=1)


CPU times: total: 1min 39s
Wall time: 1min 41s


In [ ]:
df

In [16]:
%%time
# Define columns
columns = ['Civic', 'Government', 'Hospital', 'Public', 'School', 'Sports', 'Supermarket', 'Transport']

# Define a function to apply element-wise
def transform_list(lst, val):
    return [round(val - x, 2) for x in lst]

# Use applymap to vectorize calculations
for col in columns:
    df[f'{col}_1km'] = df[f'{col}_1km'].map(lambda x: transform_list(x, 1))
    df[f'{col}_5km'] = df[f'{col}_5km'].map(lambda x: transform_list(x, 5))
    df[f'{col}_10km'] = df[f'{col}_10km'].map(lambda x: transform_list(x, 10))



CPU times: total: 2min 5s
Wall time: 2min 10s


In [17]:
%%time
# Define columns
columns = ['Civic', 'Government', 'Hospital', 'Public', 'School', 'Sports', 'Supermarket', 'Transport']

# Initialize score columns
for col in columns:
    df[f'{col}_1km_Score'] = None
    df[f'{col}_5km_Score'] = None
    df[f'{col}_10km_Score'] = None

# Define a function to calculate scores
def calculate_scores(row, columns):
    for col in columns:
        row[f'{col}_1km_Score'] = sum(row[f'{col}_1km'])
        row[f'{col}_5km_Score'] = sum(row[f'{col}_5km'])
        row[f'{col}_10km_Score'] = sum(row[f'{col}_10km'])
    return row

# Apply the function across the dataframe
df = df.apply(lambda row: calculate_scores(row, columns), axis=1)


CPU times: total: 2min 14s
Wall time: 2min 23s


In [18]:
df

,index,Building,name:en,building,building:levels,building:materials,addr:full,addr:housenumber,addr:street,addr:city,...,School_10km_Score,Sports_1km_Score,Sports_5km_Score,Sports_10km_Score,Supermarket_1km_Score,Supermarket_5km_Score,Supermarket_10km_Score,Transport_1km_Score,Transport_5km_Score,Transport_10km_Score
0,0,MexicoCityBuilding0,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,...,41.59,0.26,10.45,35.45,0.00,2.07,15.49,0.00,5.84,30.84
1,1,MexicoCityBuilding1,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,...,41.59,0.26,10.44,35.44,0.00,2.08,15.52,0.00,5.84,30.84
2,2,MexicoCityBuilding2,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,...,41.59,0.26,10.44,35.44,0.00,2.08,15.52,0.00,5.85,30.85
3,3,MexicoCityBuilding3,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,...,41.52,0.24,10.44,35.44,0.00,2.11,15.57,0.00,5.92,30.92
4,4,MexicoCityBuilding4,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,...,41.54,0.24,10.43,35.43,0.00,2.11,15.57,0.00,5.92,30.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164839,209572,MexicoCityBuilding209572,NaN,yes,1.0,NaN,NaN,NaN,NaN,NaN,...,49.16,0.00,17.76,42.76,0.00,8.25,29.92,1.26,21.10,46.10
164840,209573,MexicoCityBuilding209573,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,...,41.73,0.00,7.76,29.74,0.00,0.00,8.00,0.00,0.00,11.60
164841,209574,MexicoCityBuilding209574,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,...,41.57,0.00,8.18,30.37,0.00,0.00,8.88,0.00,0.00,12.63
164842,209577,MexicoCityBuilding209577,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,...,49.85,0.00,15.95,40.95,0.59,9.32,31.52,2.31,21.83,46.83


In [26]:
df.columns[85]

IndexError: index 85 is out of bounds for axis 0 with size 85

In [27]:
# Vectorized scaling operation
df.iloc[:, 61:85] = df.iloc[:, 61:85].apply(lambda col: (col - col.min()) / (col.max() - col.min()) * 10, axis=0)


In [28]:
# Optimized approach
try:
    df.iloc[:, 61:85] = df.iloc[:, 61:85].astype(float).round(2)
except Exception as e:
    print('Error:', e)


In [29]:
df

,index,Building,name:en,building,building:levels,building:materials,addr:full,addr:housenumber,addr:street,addr:city,...,School_10km_Score,Sports_1km_Score,Sports_5km_Score,Sports_10km_Score,Supermarket_1km_Score,Supermarket_5km_Score,Supermarket_10km_Score,Transport_1km_Score,Transport_5km_Score,Transport_10km_Score
0,0,MexicoCityBuilding0,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,...,7.07,0.52,4.18,6.74,0.00,1.63,4.11,0.00,2.41,6.26
1,1,MexicoCityBuilding1,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,...,7.07,0.52,4.18,6.74,0.00,1.64,4.11,0.00,2.41,6.26
2,2,MexicoCityBuilding2,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,...,7.07,0.52,4.18,6.74,0.00,1.64,4.11,0.00,2.41,6.27
3,3,MexicoCityBuilding3,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,...,7.04,0.48,4.18,6.74,0.00,1.66,4.13,0.00,2.44,6.28
4,4,MexicoCityBuilding4,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,...,7.05,0.48,4.18,6.74,0.00,1.66,4.13,0.00,2.44,6.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164839,209572,MexicoCityBuilding209572,NaN,yes,1.0,NaN,NaN,NaN,NaN,NaN,...,9.71,0.00,7.11,8.38,0.00,6.49,7.93,2.97,8.70,9.36
164840,209573,MexicoCityBuilding209573,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,...,7.12,0.00,3.11,5.46,0.00,0.00,2.12,0.00,0.00,2.36
164841,209574,MexicoCityBuilding209574,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,...,7.06,0.00,3.27,5.60,0.00,0.00,2.35,0.00,0.00,2.56
164842,209577,MexicoCityBuilding209577,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,...,9.95,0.00,6.39,7.98,5.96,7.33,8.36,5.45,9.01,9.51


In [30]:
df.columns

Index(['index', 'Building', 'name:en', 'building', 'building:levels',
       'building:materials', 'addr:full', 'addr:housenumber', 'addr:street',
       'addr:city', 'office', 'source', 'name:es', 'osm_id', 'osm_type',
       'ADM1_ES', 'ADM1_PCODE', 'ADM1_REF', 'ADM1ALT1ES', 'ADM1ALT2ES',
       'ADM0_ES', 'ADM0_PCODE', 'date', 'validOn', 'validTo', 'Shape_Leng',
       'Shape_Area', 'lon', 'lat', 'Civic', 'Government', 'Hospital', 'Public',
       'School', 'Sports', 'Supermarket', 'Transport', 'Civic_1km',
       'Civic_5km', 'Civic_10km', 'Government_1km', 'Government_5km',
       'Government_10km', 'Hospital_1km', 'Hospital_5km', 'Hospital_10km',
       'Public_1km', 'Public_5km', 'Public_10km', 'School_1km', 'School_5km',
       'School_10km', 'Sports_1km', 'Sports_5km', 'Sports_10km',
       'Supermarket_1km', 'Supermarket_5km', 'Supermarket_10km',
       'Transport_1km', 'Transport_5km', 'Transport_10km', 'Civic_1km_Score',
       'Civic_5km_Score', 'Civic_10km_Score', 'Govern

In [31]:
df = df[['Building', 'lon', 'lat', 'Civic_1km_Score',
       'Civic_5km_Score', 'Civic_10km_Score', 'Government_1km_Score',
       'Government_5km_Score', 'Government_10km_Score', 'Hospital_1km_Score',
       'Hospital_5km_Score', 'Hospital_10km_Score', 'Public_1km_Score',
       'Public_5km_Score', 'Public_10km_Score', 'School_1km_Score',
       'School_5km_Score', 'School_10km_Score', 'Sports_1km_Score',
       'Sports_5km_Score', 'Sports_10km_Score', 'Supermarket_1km_Score',
       'Supermarket_5km_Score', 'Supermarket_10km_Score',
       'Transport_1km_Score', 'Transport_5km_Score', 'Transport_10km_Score']]

In [32]:
df

,Building,lon,lat,Civic_1km_Score,Civic_5km_Score,Civic_10km_Score,Government_1km_Score,Government_5km_Score,Government_10km_Score,Hospital_1km_Score,...,School_10km_Score,Sports_1km_Score,Sports_5km_Score,Sports_10km_Score,Supermarket_1km_Score,Supermarket_5km_Score,Supermarket_10km_Score,Transport_1km_Score,Transport_5km_Score,Transport_10km_Score
0,MexicoCityBuilding0,-99.959145,19.174023,0.00,3.14,6.59,0.0,0.00,0.23,0.00,...,7.07,0.52,4.18,6.74,0.00,1.63,4.11,0.00,2.41,6.26
1,MexicoCityBuilding1,-99.958902,19.173881,0.00,3.14,6.59,0.0,0.00,0.24,0.00,...,7.07,0.52,4.18,6.74,0.00,1.64,4.11,0.00,2.41,6.26
2,MexicoCityBuilding2,-99.958831,19.173856,0.00,3.14,6.59,0.0,0.00,0.24,0.00,...,7.07,0.52,4.18,6.74,0.00,1.64,4.11,0.00,2.41,6.27
3,MexicoCityBuilding3,-99.957499,19.174133,0.00,3.17,6.60,0.0,0.00,0.25,0.00,...,7.04,0.48,4.18,6.74,0.00,1.66,4.13,0.00,2.44,6.28
4,MexicoCityBuilding4,-99.957630,19.174264,0.00,3.17,6.60,0.0,0.00,0.25,0.00,...,7.05,0.48,4.18,6.74,0.00,1.66,4.13,0.00,2.44,6.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164839,MexicoCityBuilding209572,-99.119749,19.683755,0.00,6.63,8.32,0.0,7.74,8.87,5.50,...,9.71,0.00,7.11,8.38,0.00,6.49,7.93,2.97,8.70,9.36
164840,MexicoCityBuilding209573,-99.869651,19.950939,1.50,6.47,8.24,0.0,0.00,2.74,0.00,...,7.12,0.00,3.11,5.46,0.00,0.00,2.12,0.00,0.00,2.36
164841,MexicoCityBuilding209574,-99.846846,19.947589,1.96,6.66,8.34,0.0,0.00,2.98,0.00,...,7.06,0.00,3.27,5.60,0.00,0.00,2.35,0.00,0.00,2.56
164842,MexicoCityBuilding209577,-99.656001,19.283160,4.78,8.99,9.50,0.0,0.00,3.42,8.72,...,9.95,0.00,6.39,7.98,5.96,7.33,8.36,5.45,9.01,9.51


In [33]:
df.to_csv('MexicoCityScoreRAW.csv', index = False)